In [1]:
# =========================================================
# 🔌 Cargar entorno y librerías
# =========================================================
%load_ext kedro.ipython

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# =========================================================
# 1️⃣ Cargar dataset desde Kedro
# =========================================================
df = catalog.load("Features_training_v2").copy()
print("Shape del dataset:", df.shape)

# =========================================================
# 2️⃣ Seleccionar variables según la hipótesis
# =========================================================
features = [
    "Monetary",
    "TimeSinceLastTxn",
    "Recency",
    "TxnCountInLast24Hours",
    "AmountZScoreByLocation"
]
target = "TransactionAmount (INR)"

df_model = df[features + [target]].dropna()
X = df_model[features]
y = df_model[target]

[10/24/25 03:35:12] INFO     Using                                                                  ]8;id=453025;file://C:\Users\Kevin\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\framework\project\__init__.py\__init__.py]8;;\:]8;id=108539;file://C:\Users\Kevin\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\framework\project\__init__.py#270\270]8;;\
                             'C:\Users\Kevin\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\                
                             framework\project\rich_logging.yml' as logging configuration.                         

                    INFO     Registered line magic '%reload_kedro'                                   ]8;id=62968;file://C:\Users\Kevin\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=934114;file://C:\Users\Kevin\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\ipython\__init__.py#63\63]8;;\

                    INFO     Registered line magic '%load_node'                                      ]8;id=767198;file://C:\Users\Kevin\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=87834;file://C:\Users\Kevin\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\ipython\__init__.py#65\65]8;;\

                    INFO     Resolved project path as:                                              ]8;id=569551;file://C:\Users\Kevin\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=879435;file://C:\Users\Kevin\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\ipython\__init__.py#180\180]8;;\
                             C:\Users\Kevin\Documents\GitHub\Ml_Bank\proyecto-ml-sebastiancarrera-k                
                             evinvivanco.                                                                          
                             To set a different path, run '%reload_kedro <project_root>'                           

[10/24/25 03:35:14] INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=315060;file://C:\Users\Kevin\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro_telemetry\plugin.py\plugin.py]8;;\:]8;id=428738;file://C:\Users\Kevin\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro_telemetry\plugin.py#243\243]8;;\
                             the product. No personal data or IP addresses are stored on our side. To              
                             opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK` environment              
                             variables, or create a `.telemetry` file in the current working                       
                             directory with the contents `consent: false`. To hide this message,                   
                             explicitly grant or deny consent. Read more at                                        
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

[10/24/25 03:35:15] INFO     Kedro project Proyecto_ML_SebastianCarrera_KevinVivanco                ]8;id=147032;file://C:\Users\Kevin\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=655714;file://C:\Users\Kevin\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\ipython\__init__.py#146\146]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=640597;file://C:\Users\Kevin\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=142558;file://C:\Users\Kevin\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\ipython\__init__.py#147\147]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=541115;file://C:\Users\Kevin\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=571870;file://C:\Users\Kevin\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\ipython\__init__.py#153\153]8;;\

[10/24/25 03:35:17] INFO     Loading data from Features_training_v2 (ParquetDataset)...        ]8;id=628691;file://C:\Users\Kevin\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=779287;file://C:\Users\Kevin\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\io\data_catalog.py#1046\1046]8;;\

Shape del dataset: (984240, 6)


In [2]:
# =========================================================
# 3️⃣ División Train/Test
# =========================================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# =========================================================
# 4️⃣ Definir modelo Random Forest + GridSearchCV
# =========================================================
rf = RandomForestRegressor(random_state=42)

param_grid = {
    "n_estimators": [100, 200],
    "max_depth": [10, 20, None],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2],
}

cv = KFold(n_splits=5, shuffle=True, random_state=42)

grid_rf = GridSearchCV(
    rf,
    param_grid=param_grid,
    cv=cv,
    scoring="r2",
    n_jobs=-1
)

# =========================================================
# 5️⃣ Entrenamiento con validación cruzada
# =========================================================
grid_rf.fit(X_train, y_train)

print("Mejores parámetros:", grid_rf.best_params_)
print("R² medio (CV):", grid_rf.best_score_)

# =========================================================
# 6️⃣ Evaluación final
# =========================================================
best_rf = grid_rf.best_estimator_

y_pred = best_rf.predict(X_test)

r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)

print("\n===== Métricas del Modelo Random Forest =====")
print(f"R²: {r2:.3f}")
print(f"MSE: {mse:.3f}")
print(f"RMSE: {rmse:.3f}")
print(f"MAE: {mae:.3f}")

# =========================================================
# 7️⃣ Gráficos de evaluación
# =========================================================

# 🎯 a) Valores reales vs predichos
plt.figure(figsize=(6,6))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()],
         [y_test.min(), y_test.max()],
         color='red', linestyle='--', label='Línea ideal')
plt.title("Valores Reales vs Predichos – Random Forest")
plt.xlabel("Valores reales (y_test)")
plt.ylabel("Predicciones (y_pred)")
plt.legend()
plt.grid(True)
plt.show()

# 🎯 b) Distribución de residuos
residuals = y_test - y_pred
plt.figure(figsize=(6,5))
sns.histplot(residuals, bins=30, kde=True)
plt.title("Distribución de Residuos – Random Forest")
plt.xlabel("Error (y_real - y_pred)")
plt.ylabel("Frecuencia")
plt.grid(True)
plt.show()

# 🎯 c) Importancia de variables
importances = best_rf.feature_importances_
imp_df = pd.DataFrame({"Variable": features, "Importancia": importances})
imp_df = imp_df.sort_values(by="Importancia", ascending=False)

plt.figure(figsize=(8,4))
sns.barplot(data=imp_df, x="Importancia", y="Variable", palette="Greens_r")
plt.title("Importancia de Variables – Random Forest")
plt.xlabel("Importancia relativa")
plt.ylabel("")
plt.grid(True)
plt.show()

print("\nImportancia de variables:")
print(imp_df)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:33                                                                                   │
│                                                                                                  │
│   30 # =========================================================                                 │
│   31 # 5️⃣ Entrenamiento con validación cruzada                                                    │
│   32 # =========================================================                                 │
│ ❱ 33 grid_rf.fit(X_train, y_train)                                                               │
│   34                                                                                             │
│   35 print("Mejores parámetros:", grid_rf.best_params_)                                          │
│   36 print("R² medio (CV):", grid_rf.best_score_)                                                │
│                                                                                                  │
│ C:\Users\Kevin\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\sklearn\base.py:1365 in wrapper   │
│                                                                                                  │
│   1362 │   │   │   │   │   prefer_skip_nested_validation or global_skip_validation               │
│   1363 │   │   │   │   )                                                                         │
│   1364 │   │   │   ):                                                                            │
│ ❱ 1365 │   │   │   │   return fit_method(estimator, *args, **kwargs)                             │
│   1366 │   │                                                                                     │
│   1367 │   │   return wrapper                                                                    │
│   1368                                                                                           │
│                                                                                                  │
│ C:\Users\Kevin\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\sklearn\model_selection\_search.p │
│ y:1051 in fit                                                                                    │
│                                                                                                  │
│   1048 │   │   │   │                                                                             │
│   1049 │   │   │   │   return results                                                            │
│   1050 │   │   │                                                                                 │
│ ❱ 1051 │   │   │   self._run_search(evaluate_candidates)                                         │
│   1052 │   │   │                                                                                 │
│   1053 │   │   │   # multimetric is determined here because in the case of a callable            │
│   1054 │   │   │   # self.scoring the return type is only known after calling                    │
│                                                                                                  │
│ C:\Users\Kevin\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\sklearn\model_selection\_search.p │
│ y:1605 in _run_search                                                                            │
│                                                                                                  │
│   1602 │                                                                                         │
│   1603 │   def _run_search(self, evaluate_candidates):                                           │
│   1604 │   │   """Search all candidates in param_grid"""                                         │
│ ❱ 1605 │   │   evaluate_candidates(ParameterGrid(self.param_grid))                               │
│   1606                                                   